# Intensity Calculation

In [3]:
import utils 
import torch
import os
import pickle
import csv
import torch.nn.functional as F
import argparse
import utils
import sys
import os
import datetime
import time
from torch.utils.data import DataLoader
import conttime
import torch
import matplotlib.pyplot as plt
import pdb
import re


# Data

In [34]:
data_path = f'data/conttime_100_Seq_2_Dim_100_MaxLen_8_LSTMUnits.pkl'
dim_process = 2

In [35]:
interarrival_seqs, event_types_seqs, length_seqs = utils.open_pkl_file(data_path, 'test')

In [36]:
# Padding # Do not need to for NN Calc
interarrival_seqs, event_types_seqs = utils.padding_full(
        interarrival_seqs, 
        event_types_seqs, 
        length_seqs, 
        dim_process
    )

In [37]:
batch_size = 1
interarrival_seqs = interarrival_seqs[0:batch_size]
event_types_seqs = event_types_seqs[0:batch_size]
length_seqs = length_seqs[0:batch_size]

In [49]:
interarrival_seqs.shape

torch.Size([1, 100])

# Generating Simulated Data

In [42]:
sim_diffs_seqs, sim_diffs_index, total_duration_seqs = utils.generate_simulation(interarrival_seqs, length_seqs)

In [48]:
sim_diffs_seqs.shape

torch.Size([1, 430])

# Neural Estimated Intensity

In [9]:
model = torch.load('./models/conttime_100_Seq_2_Dim_100_MaxLen_8_LSTMUnits.pt')

# calculating gates at event times

In [45]:
# Calculates hidden state from sequence (forward)
hidden_states, cell_gates, cell_bar_gates, decay_gates, output_gates = model.forward(event_types_seqs, interarrival_seqs)

In [47]:
hidden_states.shape

torch.Size([99, 1, 8])

# Calculating at random times

In [50]:
# Initalize simulated gate values
sim_cell_gates = []
sim_cell_bar_gates = []
sim_decay_gates = []
sim_output_gates = []

In [51]:
# Looping over sequences in batch
for idx in range(batch_size):

    # Finds value of gates of the original sequences at simulated index points???
    layer_cell_gates = cell_gates[sim_diffs_index[idx], idx, :]
    sim_cell_gates.append(layer_cell_gates)

    layer_cell_bar_gates = cell_bar_gates[sim_diffs_index[idx], idx, :]
    sim_cell_bar_gates.append(layer_cell_bar_gates)

    layer_decay_gates = decay_gates[sim_diffs_index[idx], idx, :]
    sim_decay_gates.append(layer_decay_gates)

    layer_output_gates = output_gates[sim_diffs_index[idx], idx, :]
    sim_output_gates.append(layer_output_gates)
    
# Transforms sim_gates from a list of length batch_size with each object made up of sim_len lists of lstm_units to a tensor with dimensions sim_len x batch_size x lstm_units
sim_cell_gates = torch.stack(sim_cell_gates).transpose(0,1)
sim_cell_bar_gates = torch.stack(sim_cell_bar_gates).transpose(0,1)
sim_decay_gates = torch.stack(sim_decay_gates).transpose(0,1)
sim_output_gates = torch.stack(sim_output_gates).transpose(0,1)

sim_hidden_list = []

In [54]:
# Looping over each simulated event
for idx in range(sim_diffs_seqs.shape[1]):
    # Calculates simulated hidden state at the idx^th event for each sequence in batch
    _, sim_hidden_t__i = model.lstm_cell.decay(sim_cell_gates[idx], 
                                                sim_cell_bar_gates[idx],
                                                sim_decay_gates[idx], 
                                                sim_output_gates[idx], 
                                                sim_diffs_seqs[:, idx])

    sim_hidden_list.append(sim_hidden_t__i)            

# Transforms hidden_list from a list of length sim_len with each object made up of batch_size lists of lstm_units to a tensor with dimensions sim_len x batch_size x lstm_units
sim_hidden_list = torch.stack(sim_hidden_list)        

# Calculates simulated intensity at each simulated event time
sim_intensity = torch.nn.functional.softplus(model.hidden_lambda(sim_hidden_list)).transpose(0,1)

In [ ]:
hidden_states, *_ = model(event_types_seqs, interarrival_seqs)

In [ ]:
lambda_all = F.softplus(model.hidden_lambda(hidden_states))

# Calculating Intensity Neural (Actual)

In [4]:
with open("./gen_models/conttime_100_Seq_2_Dim_100_MaxLen_8_LSTMUnits.pkl", "rb") as f:
    # Load the data from the pickle file
    data = pickle.load(f)

In [5]:
data

{'scale': array([1., 1.]),
 'W_alpha': array([[ 0.8592322 , -0.3672489 ],
        [-0.6321624 , -0.59087944],
        [ 0.13545007,  0.1910894 ],
        [ 0.92902905,  0.3063542 ],
        [ 0.49781328,  0.30713975],
        [ 0.4954296 ,  0.9226135 ],
        [-0.9832234 , -0.7871112 ],
        [-0.40259257,  0.31282237]], dtype=float32),
 'Emb_event': array([[ 0.6196251 ,  0.7443518 ,  0.9292952 ,  0.4473707 ,  0.28495064,
          0.43490723, -0.06480198, -0.34883064],
        [-0.12071079,  0.45937815,  0.9880292 ,  0.35374743,  0.581645  ,
         -0.6581715 , -0.94630146,  0.6007405 ],
        [ 0.80744505, -0.9506476 , -0.01650536,  0.05251034,  0.19273202,
         -0.8960849 ,  0.7901791 ,  0.45653236]], dtype=float32),
 'W_recur': array([[ 6.36700034e-01,  4.45505662e-04,  6.20378792e-01,
         -8.08062971e-01, -5.62099934e-01, -4.82561886e-01,
         -6.37884885e-02, -8.12535957e-02,  4.19019550e-01,
         -6.43894017e-01,  6.28997684e-02, -6.64515555e-01,
       

In [13]:
import torch
import torch.nn as nn
import numpy as np

# Define your custom weights as a numpy array
custom_weights_np = np.array([[ 0.8592322 , -0.3672489 ],
                              [-0.6321624 , -0.59087944],
                              [ 0.13545007,  0.1910894 ],
                              [ 0.92902905,  0.3063542 ],
                              [ 0.49781328,  0.30713975],
                              [ 0.4954296 ,  0.9226135 ],
                              [-0.9832234 , -0.7871112 ],
                              [-0.40259257,  0.31282237]], dtype=np.float32)

# Transpose the numpy array
custom_weights_np_transposed = custom_weights_np.T

# Convert the transposed numpy array to a PyTorch tensor
custom_weights_tensor = torch.tensor(custom_weights_np_transposed)

# Define the linear layer
input_features = custom_weights_np_transposed.shape[1]
output_features = custom_weights_np_transposed.shape[0]
linear_layer = nn.Linear(input_features, output_features, bias=False)

# Assign custom weights
with torch.no_grad():  # To avoid tracking this operation in autograd
    linear_layer.weight.copy_(custom_weights_tensor)
    linear_layer.weight.requires_grad_(False)  # Detach the weights from gradient computation


In [14]:
input_features

8

In [15]:
linear_layer.weight

Parameter containing:
tensor([[ 0.8592, -0.6322,  0.1355,  0.9290,  0.4978,  0.4954, -0.9832, -0.4026],
        [-0.3672, -0.5909,  0.1911,  0.3064,  0.3071,  0.9226, -0.7871,  0.3128]])

In [16]:
model.hidden_lambda.weight

Parameter containing:
tensor([[-0.2137,  0.2984, -0.0158, -0.1711,  0.4153, -0.3671, -0.4657, -0.1532],
        [-0.2258, -0.0359, -0.2750, -0.0398, -0.2373, -0.0583, -0.0530,  0.5120]],
       requires_grad=True)

In [ ]:
gen_model = seq_generator.NeuralHawkesCTLSTM(generator_settings)

for time in sequence:
    

# Data for R

In [7]:
interarrival_seqs[0:2]

[tensor([0.2480, 0.6965, 0.0386, 0.2882, 0.1562, 0.3219, 0.2465, 0.3069, 0.0895,
         0.2165, 0.3193, 0.1113, 0.0127, 0.3950, 0.1227, 0.3190, 0.2185, 0.0250,
         0.0506, 0.0367, 0.0552, 0.0735, 0.0135, 0.0265, 0.0041, 0.0851, 0.2348,
         0.1769, 0.0548, 0.0146, 0.2199, 0.1452, 0.2108, 0.0197, 0.2057, 0.0272,
         0.0598, 0.0933, 0.1361, 0.5554, 0.2824, 0.0402, 0.0202, 0.3998, 0.0644,
         0.3322, 0.0838, 0.0805]),
 tensor([0.2725, 0.0124, 0.0933, 0.0879, 0.1025, 0.0491, 0.0934, 0.2177, 0.3190,
         0.0821, 0.1706, 0.1492, 0.3024, 1.1722, 0.0316, 0.0770, 0.0460, 0.4269,
         0.6420, 0.0486, 0.3461, 0.2229, 0.1622])]

In [19]:
# Specify the file path
file_path = 'length_seqs_train.csv'

# Write data to CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(length_seqs)

In [29]:
interarrival_seqs, event_types_seqs = utils.padding_full(
        interarrival_seqs, 
        event_types_seqs, 
        length_seqs, 
        dim_process
    )

In [26]:
torch.save(interarrival_seqs, 'interarrival_seqs.pt')

In [27]:
torch.load('interarrival_seqs.pt')

tensor([[0.0000, 0.2480, 0.6965,  ..., 0.0805, 0.0000, 0.0000],
        [0.0000, 0.2725, 0.0124,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.1545, 0.1190,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.1723, 0.2842,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0263, 0.3160,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.2332, 0.0590,  ..., 0.0000, 0.0000, 0.0000]])

In [ ]:
# Load the model
model = torch.load("conttime_100_Seq_2_Dim_100_MaxLen_8_LSTMUnits.pt")

# Print the model
print(model)
